In [ ]:
i = 0 #step vis graph
path = 0
while cap.isOpened():
    ret, frame = cap.read()
    
    if thymio.get_state() == 0:
        thymio.initialisation_step(img, margin, True)
        thymio.set_state(1)
        path = thymio.get_path()
    
    if thymio.get_state() == 1:  #motion control commence
        pos_thy, two_centres = vs.detect_start(frame, False, False)  #recupere pos thymio
        frame = glb.draw_thymio(frame, pos_thy)   # dessine point thymio new fenetre
        frame = glb.draw_path(frame, thymio.get_path()) #dessine chemin du thymio new fenetre
        
        thymio.update_position_cam(two_centres)  #update pos thymio et angle dans la classe robot
        if len(path) > (i+1):
            next_goal = (path[i+1].x,path[i+1].y)
            # astolfi implemetation
            angle = get_angle2goal(thymio.get_geometry(), [1,0])  #angle entre thymio et point
            temp=ctrl.astolfi(thymio.get_geometry(), angle, next_goal, node)
            if temp==1:
                i=i+1
            # jank implementation that works
        else : 
            ctrl.stop_motors(node)
        '''
            angle = get_angle2goal(thymio.get_geometry(), next_goal)  #angle entre thymio et point
            get_correct_orientation(angle, node, 50, 0.2)
            
            if get_correct_orientation(angle, node, 50, 0.2):
                print("avance")
                
                
                get_distance(thymio.get_geometry(), next_goal, 30, 200, node)
                
                if get_distance(thymio.get_geometry(), next_goal, 30, 200, node):
                    i = i + 1
                    if i == (len(path)-1):
                        thymio.set_state(2)
                        ctrl.stop_motors(node)
                        print("arrivee")
                        print(thymio.get_state())
                        break      
        '''
        
    cv2.imshow('frame',frame)     
    if cv2.waitKey(1) == ord('q'):
        ctrl.stop_motors(node)
        break
        
cap.release()
cv2.destroyAllWindows()